## HyperLang Sample Concept


In this sample concept, we will generate transcripts in Spanish given the country of origin and scenario. For now, we manually specify the country and scenario in this notebook, but, in the future we can create more
specific parameters (e.g. difficulty, number of speakers in conversation) that will be inputted by the end user.

### Helper Classes

In [18]:
"""
Module for handling text generation using Hugging Face's API and for providing
a teacher agent that can annotate or translate a given Spanish conversation.
"""

import os
import asyncio
import config
from typing import Any, AsyncGenerator
from dotenv import load_dotenv
from huggingface_hub import InferenceClient

load_dotenv()


class PromptHandler:
    """
    Loads and formats prompt templates from a file.
    """

    def __init__(self, template_path: str) -> None:
        self.template_path = template_path

    def load_prompt(self) -> str:
        with open(self.template_path, 'r', encoding='utf-8') as file:
            return file.read()

    def format_prompt_scenario(self, scenario: str, country_name: str = "Spain") -> str:
        template = self.load_prompt()
        return template.format(
            scenario=scenario,
            country_name=country_name
        )

    def format_prompt(self, **kwargs) -> str:
        template = self.load_prompt()
        return template.format(**kwargs)


class TranscriptGenerator:
    """
    Generates or streams text from a Hugging Face model.
    """

    def __init__(self, model_name: str, temperature: float = 0.7) -> None:
        self.client = InferenceClient(api_key=os.getenv('HUGGINGFACE_TOKEN'))
        self.model_name = model_name
        self.temperature = temperature

    def generate_transcript(self, prompt: str) -> str:
        try:
            messages = [{"role": "user", "content": prompt}]
            completion = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=self.temperature,
                max_tokens=10000
            )
            return completion.choices[0].message.content
        except Exception as exc:
            return f"Error: {str(exc)}"

    async def generate_transcript_stream(
        self, prompt: str
    ) -> AsyncGenerator[str, None]:
        try:
            messages = [{"role": "user", "content": prompt}]
            stream = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=self.temperature,
                max_tokens=10000,
                stream=True
            )
            for chunk in stream:
                content = chunk.choices[0].delta.content
                if content:
                    yield content
                    await asyncio.sleep(0.01)
        except Exception as exc:
            yield f"Error: {str(exc)}"


class TeacherAgent:
    """
    Uses a teacher-specific prompt to provide annotated explanations or translations.
    """

    def __init__(
        self,
        template_path: str,
        model_name: str,
        temperature: float = 0.7
    ) -> None:
        self.prompt_handler = PromptHandler(template_path)
        self.transcript_generator = TranscriptGenerator(
            model_name=model_name,
            temperature=temperature
        )

    def explain_transcript(
        self,
        transcript: str,
        scenario: str,
        country_name: str = "Spain"
    ) -> str:
        teacher_prompt = self.prompt_handler.format_prompt(
            transcript=transcript,
            scenario=scenario,
            country_name=country_name
        )
        return self.transcript_generator.generate_transcript(teacher_prompt)

    async def explain_transcript_stream(
        self,
        transcript: str,
        scenario: str,
        country_name: str = "Spain"
    ) -> AsyncGenerator[str, None]:
        teacher_prompt = self.prompt_handler.format_prompt(
            transcript=transcript,
            scenario=scenario,
            country_name=country_name
        )
        async for chunk in self.transcript_generator.generate_transcript_stream(teacher_prompt):
            yield chunk

In [19]:
country_name = "Colombia" ## specify the country (optional)

In [20]:
scenario_path = "./data/scenarios/cafe.txt"
with open(scenario_path, 'r') as file:
    scenario = file.read()
print(scenario)

A tourist enters a busy cafe and goes up to the counter to order.
They don't know exactly what to order and want some advice from the server.
After ordering and paying with card they take a seat and wait for their order to arrive.
Once it arrives they interact with the server and finsih their order and then leave.


In [21]:
print(config.PROMPT_TEMPLATE_PATH)
prompt_handler = PromptHandler(config.PROMPT_TEMPLATE_PATH)
prompt = prompt_handler.format_prompt_scenario(scenario, country_name=country_name)

./data/system_prompt_template.txt


In [22]:
print(prompt)

You are an expert content creator for Spanish language learners. Your role is to produce a natural-sounding, Spanish-only conversation between two characters. The conversation should be based on the following details:

- Country: Colombia
- Scenario: A tourist enters a busy cafe and goes up to the counter to order.
They don't know exactly what to order and want some advice from the server.
After ordering and paying with card they take a seat and wait for their order to arrive.
Once it arrives they interact with the server and finsih their order and then leave.

Requirements:

1. Spanish Only
   - Do not include any English text.
   - Reflect the local dialect and culture of Colombia in your Spanish.

2. Use Personal Names
   - Use realistic names (e.g., Ana, David, Camila) for each character depending on the context.
   - Avoid generic labels like “Visitor,” “Guide,” “Man,” or “Woman.”

3. No Background Text or Meta Commentary
   - Do not include narration, scene descriptions, or notes

## Synchronous Output

In [23]:
transcript_generator = TranscriptGenerator(model_name=config.MODEL_NAME)
initial_transcript = transcript_generator.generate_transcript(prompt)

## Asynchronous Output

In [24]:
async def stream_convo():
    async def convo_stream():
        async for token in transcript_generator.generate_transcript_stream(prompt):
            yield token
    async for token in convo_stream():
        print(token, end='', flush=True)

In [25]:
await stream_convo()

 Carlos (el barista): ¡Hola! ¿Qué podemos hacer por ti hoy?

Juan (el turista): Hola, ¿tienes alguna recomendación para un desayuno? No conozco mucho la comida aquí.

Carlos: ¡Claro! Siempre me alegra ayudar a los visitantes. ¿Te gustan los arepas? Son una delicia típica de Colombia.

Juan: ¡Por supuesto! ¿Y qué puedo acompañarlas?

Carlos: Puedes elegir entre el café, el jugo de frutas o un zumo de tamarindo.

Juan: ¡Entiendo! ¿Yo puedo ordenar un café con leche y una arepa con queso, por favor.

Carlos: ¡Claro, señor! ¿Puedes pagar con tarjeta de crédito?

Juan: Sí, ¡claro!

Carlos: ¡Gracias por su compra! Su orden estará lista pronto.

(Juan se sienta en una mesa y espera su orden)

Carlos (entregando la orden): Aquí está tu desayuno, señor. ¡Disfruta!

Juan: ¡Muchísimas gracias! ¿Tienes alguna otra recomendación para hoy?

Carlos: ¡Sí, claro! Podrías probar nuestros postres típicos, como el buñuelo o el arequipe.

Juan: ¡Gracias por la sugerencia! ¡Hasta pronto!

Carlos: ¡Hasta lue

## Adding a Teacher

In [26]:
teacher_agent = TeacherAgent(
    template_path=config.TEACHER_TEMPLATE_PATH,
    model_name=config.MODEL_NAME,
    temperature=0.2
)

In [27]:
teacher_response = teacher_agent.explain_transcript(transcript=initial_transcript,
    scenario=scenario,
    country_name=country_name)

In [28]:
print("=== Initial Transcript ===")
print(initial_transcript)
print("\n=== Teacher Explanation ===")
print(teacher_response)

=== Initial Transcript ===
Juan Pablo (el turista): Hola, ¿puedes darme alguna recomendación sobre lo que ordenar?

Ana (el waitress): ¡Claro, qué deseas probar hoy? Tenemos un burrito con carne de res, un arepa con queso y una torta de guayaba.

Juan Pablo: Eso sí, ¿cómo es el burrito con carne?

Ana: Es uno de nuestros platos más populares. Tiene carne de res, arroz, frijoles negros y salsa verde.

Juan Pablo: Perfecto, lo tomaré. ¿Y si me lo traes con un refresco de piña?

Ana: Claro, nosotros también tenemos un refresco de maracuyá. ¿Te gustaría probarlo?

Juan Pablo: ¡Claro! Lo pediré, entonces.

Ana: Muy bien. ¡Gracias por elegir nuestro café!

[Juan Pablo paga con su tarjeta de crédito y se sienta a esperar su pedido.]

Ana (el waitress): Aquí está tu pedido, señor Juan Pablo. Disfruta!

Juan Pablo (el turista): Muchas gracias, Ana. ¡Espero volver pronto!

Ana (el waitress): Es un placer tenerte aquí, ¡vuelve siempre!

[Juan Pablo comienza a comer y disfrutar su pedido. Una vez 

In [29]:
async def stream_convo():
    async def convo_stream():
        async for token in teacher_agent.explain_transcript_stream(transcript=initial_transcript,
    scenario=scenario,
    country_name=country_name
):
            yield token
    async for token in convo_stream():
        print(token, end='', flush=True)

In [30]:
await stream_convo()

 -----
Juan Pablo (the tourist): Hola, ¿puedes darme alguna recomendación sobre lo que ordenar?

Ana (the waitress): ¡Claro, qué deseas probar hoy? Tenemos un burrito con carne de res, un arepa con queso y una torta de guayaba.

**Maestro**: Juan Pablo is asking for a recommendation on what to order. Ana is offering three options: a beef burrito, a cheese arepa, and a guava tart.

Juan Pablo: Eso sí, ¿cómo es el burrito con carne?

Ana: Es uno de nuestros platos más populares. Tiene carne de res, arroz, frijoles negros y salsa verde.

**Maestro**: Juan Pablo wants to know about the beef burrito. Ana explains that it's one of their most popular dishes, containing beef, rice, black beans, and green sauce.

Juan Pablo: Perfecto, lo tomaré. ¿Y si me lo traes con un refresco de piña?

Ana: Claro, nosotros también tenemos un refresco de maracuyá. ¿Te gustaría probarlo?

Juan Pablo: ¡Claro! Lo pediré, entonces.

Ana: Muy bien. ¡Gracias por elegir nuestro café!

**Maestro**: Juan Pablo orders 